python (flask)

For implementing Google Sign-In in a Flask application, the steps to create credentials and configure Google OAuth are similar to those for React.js and Next.js. However, Flask acts as both the backend and handles routing, so you will integrate the OAuth flow differently. Here's the complete process:

---

### **Step 1: Set Up Google Cloud Project**
Follow these steps to create the required credentials in the Google Cloud Console:

1. **Go to the [Google Cloud Console](https://console.cloud.google.com/)**.
2. **Create a Project**:
   - Click on the project dropdown and select **New Project**.
   - Provide a name for your project and click **Create**.
3. **Enable Google APIs**:
   - Navigate to **APIs & Services** > **Library**.
   - Search for **Google Identity Services API** or **Google+ API** and enable it.
4. **Configure OAuth Consent Screen**:
   - Go to **APIs & Services** > **OAuth Consent Screen**.
   - Select the user type (usually **External**).
   - Fill in details like app name, user support email, and developer contact information.
   - Save and continue through the steps.
5. **Create OAuth Credentials**:
   - Go to **APIs & Services** > **Credentials**.
   - Click **Create Credentials** > **OAuth 2.0 Client IDs**.
   - Choose **Web Application** as the application type.
   - Add the following:
     - **Authorized JavaScript Origins**: `http://localhost:5000` (Flask app's local URL).
     - **Authorized Redirect URIs**: `http://localhost:5000/login/callback`.
   - Click **Create** and copy the `CLIENT_ID` and `CLIENT_SECRET`.

---

### **Step 2: Install Required Libraries**
Use `pip` to install the necessary Python libraries:

```bash
pip install Flask google-auth-oauthlib
```

---

### **Step 3: Flask Application Code**
Here's how to set up Google OAuth in Flask:

```python
from flask import Flask, redirect, url_for, session, request
from google_auth_oauthlib.flow import Flow
import os
from pip._vendor import cachecontrol
import google.auth.transport.requests
import requests

# Flask app setup
app = Flask(__name__)
app.secret_key = os.environ.get("FLASK_SECRET_KEY", "supersecretkey")

# Google OAuth 2.0 setup
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # For development only
GOOGLE_CLIENT_ID = "your-client-id"
GOOGLE_CLIENT_SECRET = "your-client-secret"
REDIRECT_URI = "http://localhost:5000/login/callback"

# Initialize OAuth flow
flow = Flow.from_client_config(
    {
        "web": {
            "client_id": GOOGLE_CLIENT_ID,
            "project_id": "your-project-id",
            "auth_uri": "https://accounts.google.com/o/oauth2/auth",
            "token_uri": "https://oauth2.googleapis.com/token",
            "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
            "client_secret": GOOGLE_CLIENT_SECRET,
            "redirect_uris": [REDIRECT_URI],
        }
    },
    scopes=["https://www.googleapis.com/auth/userinfo.profile", "https://www.googleapis.com/auth/userinfo.email", "openid"],
)

@app.route("/")
def index():
    return "Welcome to Google Sign-In with Flask!"

@app.route("/login")
def login():
    authorization_url, state = flow.authorization_url()
    session["state"] = state
    return redirect(authorization_url)

@app.route("/login/callback")
def login_callback():
    flow.fetch_token(authorization_response=request.url)

    if not session["state"] == request.args["state"]:
        return "Request state does not match!", 400

    credentials = flow.credentials
    request_session = requests.Session()
    cached_session = cachecontrol.CacheControl(request_session)
    token_request = google.auth.transport.requests.Request(session=cached_session)

    id_info = google.oauth2.id_token.verify_oauth2_token(
        credentials.id_token, token_request, GOOGLE_CLIENT_ID
    )

    # Use id_info to get user details
    user_email = id_info.get("email")
    return f"Hello, {user_email}!"

if __name__ == "__main__":
    app.run(debug=True)
```

---

### **Step 4: Save Credentials Securely**
- Store `GOOGLE_CLIENT_ID` and `GOOGLE_CLIENT_SECRET` in a `.env` file:
  ```env
  FLASK_SECRET_KEY=your-flask-secret-key
  GOOGLE_CLIENT_ID=your-client-id
  GOOGLE_CLIENT_SECRET=your-client-secret
  ```

- Use the `python-dotenv` library to load environment variables:
  ```bash
  pip install python-dotenv
  ```

- Update your Flask app:
  ```python
  from dotenv import load_dotenv
  load_dotenv()
  GOOGLE_CLIENT_ID = os.getenv("GOOGLE_CLIENT_ID")
  GOOGLE_CLIENT_SECRET = os.getenv("GOOGLE_CLIENT_SECRET")
  ```

---

### **Step 5: Run the Flask Application**
1. Start the Flask app:
   ```bash
   flask run
   ```
2. Visit `http://localhost:5000` in your browser.
3. Click the **Sign in with Google** button to test the OAuth flow.

---

### **Step 6: Production Deployment**
1. Add your production domain (e.g., `https://your-domain.com`) to the **Authorized JavaScript Origins** and **Redirect URIs** in the Google Cloud Console.
2. Use HTTPS in production for secure communication.

---

This setup ensures a seamless and secure integration of Google Sign-In with your Flask application!